In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import GradientBoostingRegressor


In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

In [ ]:
test_df.head()

In [ ]:
train_df.head()

In [ ]:
full_df = pd.concat([train_df, test_df]).reset_index(
    drop=True).drop(['PassengerId'], axis=1)


In [ ]:
full_df

In [ ]:
full_df.info()

In [ ]:
miss_data = full_df[['Age', 'Fare', 'Cabin', 'Embarked']]

In [ ]:
miss_data.value_counts()

In [ ]:
miss_data.describe()

In [ ]:
print(f'''
    Count of Age missings: {miss_data.Age.isna().sum()} or {round(miss_data.Age.isna().sum() * 100 / len(miss_data), 2)} %
    Count of Fare missings: {miss_data.Fare.isna().sum()} or {round(miss_data.Fare.isna().sum() * 100 / len(miss_data), 2)}
    Count of Cabin missings: {miss_data.Cabin.isna().sum()} or {round(miss_data.Cabin.isna().sum() * 100 / len(miss_data), 2)} %
    Count of Embarked missings: {miss_data.Embarked.isna().sum()} or {round(miss_data.Embarked.isna().sum() * 100 / len(miss_data), 2)} %
    ''')


In [ ]:
full_df[full_df.Fare == 0]

In [ ]:
full_df[full_df.Fare == 0].describe()

In [ ]:
full_df.Embarked.fillna(full_df.Embarked.mode()[0], inplace = True)
full_df.Fare.fillna(full_df.Fare.mode()[0], inplace = True)
full_df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [ ]:
sns.displot(data=full_df, x='Fare')

In [ ]:
Fare_class = pd.qcut(full_df.Fare, q=8, labels=[1, 2, 3, 4, 5, 6, 7, 8])
full_df['Fare_class'] = Fare_class.astype(int)
sns.displot(data=full_df, x='Fare_class', hue='Survived')

In [ ]:
full_df.corr()

In [ ]:
full_df['Title']=0
full_df['Title']=full_df.Name.str.extract('([A-Za-z]+)\.') 
full_df['Title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col',
                         'Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)


In [ ]:
full_df

In [ ]:
sns.catplot(data=full_df, y='Age', x='Title', kind='bar')

In [ ]:
full_df.isna().sum()

In [ ]:
full_df.drop(['Age'], axis=1).corrwith(other=full_df.Age)

In [ ]:
full_df.Age.describe()

In [ ]:
sns.catplot(data=full_df, y='Age', x='Pclass', kind='bar')
sns.catplot(data=full_df, y='Age', x='SibSp', kind='bar')
sns.catplot(data=full_df, y='Age', x='Parch', kind='bar')
sns.catplot(data=full_df, y='Age', x='Fare_class', kind='bar')

In [ ]:
le = LabelEncoder()
categorical_features = ['Sex', 'Title', 'Embarked']
full_df_enc = full_df.copy()
for feature  in categorical_features: 
    feature_enc = le.fit_transform(full_df[feature])
    full_df_enc[feature] = pd.Series(feature_enc.reshape(-1,))


In [ ]:
full_df_enc

In [ ]:
sns.heatmap(data=full_df_enc.corr(), annot=True)


In [ ]:
sns.heatmap(data=full_df_enc.corr(), annot=True)


In [ ]:
full_df_enc.loc[full_df_enc['Title'] == 0, 'Title'] = 4


In [ ]:
full_df_enc.drop(['Age'], axis=1).corrwith(other=full_df_enc.Age)


In [ ]:
X = full_df_enc[full_df_enc['Age'].notna()].drop(['Age', 'Name', 'Survived'], axis=1)
y = full_df_enc[full_df_enc['Age'].notna()].Age
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=X['Title'])


In [ ]:
regr_3 = GradientBoostingRegressor(random_state=0, max_depth=4, n_estimators=1200, 
                                   learning_rate=0.018, loss='huber', warm_start=True, verbose=1)

regressors = [regr_3]
for regressor in regressors:
    pipe = Pipeline([('scaler', StandardScaler()), ('regr', regressor)])
    pipe.fit(X_train, y_train)

    print(regressor)
    print(pipe.score(X_train, y_train))
    print(f' test_score: {pipe.score(X_test, y_test)}')
  

In [ ]:
pd.Series(pipe.predict(X_test)-y_test).describe()


In [ ]:
full_df_enc.info()

In [ ]:
predicted_age = pipe.predict(full_df_enc[full_df_enc['Age'].isna()].drop(['Name', 'Survived', 'Age'], axis=1))


In [ ]:
full_df_enc.loc[full_df_enc['Age'].isna(), 'Age'] = np.round(predicted_age, 1)


In [ ]:
full_df_enc.Age.hist()


In [ ]:
full_df.Age.hist()


In [ ]:
X_sur = full_df_enc.loc[full_df_enc.Survived.notna()].drop(['Survived', 'Name'], axis=1)  
y_sur = np.array(full_df_enc.loc[full_df_enc.Survived.notna()]['Survived'].astype(int))
X_train_sur, X_test_sur, y_train_sur, y_test_sur = train_test_split(X_sur, y_sur, test_size=0.4, random_state = 0, stratify=X_sur['Title'])


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, PassiveAggressiveClassifier
from sklearn.metrics import confusion_matrix
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, IsolationForest, RandomForestClassifier, RandomTreesEmbedding, StackingClassifier
from sklearn.ensemble import VotingClassifier
# from hpbandster_sklearn import HpBandSterSearchCV


In [ ]:
classifier_10 = GradientBoostingClassifier(random_state=0, criterion='friedman_mse',
                                           learning_rate=0.18,
                                           loss='exponential',
                                           max_depth=5,
                                           max_leaf_nodes=30,
                                           min_samples_leaf=3,
                                           min_samples_split=20,
                                           subsample=0.9736842105263157,
                                           n_estimators=22)


In [ ]:
classifiers = [classifier_10]
for classifier in classifiers:
    classifier.fit(X_train_sur, y_train_sur)
    print(classifier)
    print(f'train score: {classifier.score(X_train_sur, y_train_sur)}')
    print(f'test score: {classifier.score(X_test_sur, y_test_sur)}')


In [ ]:
confusion_matrix(y_test_sur, classifier_10.predict(X_test_sur))


In [ ]:
predicted_survived = classifier_10.predict(full_df_enc.loc[full_df_enc.Survived.isna()].drop(['Name', 'Survived'], axis=1))


In [ ]:
predicted_survived


In [ ]:
test_df['Survived'] = predicted_survived.astype(int)


In [ ]:
submission = pd.DataFrame(index=[test_df.PassengerId, test_df.Survived])



In [ ]:
submission.to_csv('submission.csv')
